In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

class SVM():
    def __init__(self, kernel='linear', degree=3, C=1.0, tol=1e-3, max_iter=1000):
        self.kernel = kernel
        self.degree = degree
        self.C = C
        self.tol = tol
        self.max_iter = max_iter

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.X = X
        self.y = y

        # Initialize alpha and bias
        self.alpha = np.zeros(n_samples)
        self.b = 0

        # Training loop
        num_changed_alphas = 0
        examine_all = True

        for _ in range(self.max_iter):
            num_changed_alphas = 0

            if examine_all:
                # Loop over all training examples
                for i in range(n_samples):
                    num_changed_alphas += self.examine(i)
            else:
                # Loop over examples where alpha is not 0 and not C
                non_bound_indices = np.where((self.alpha > 0) & (self.alpha < self.C))[0]
                for i in non_bound_indices:
                    num_changed_alphas += self.examine(i)

            if examine_all:
                examine_all = False
            elif num_changed_alphas == 0:
                examine_all = True

        # Compute the bias
        self.b = self._compute_bias()

    def _kernel_function(self, x1, x2):
        if self.kernel == 'linear':
            return np.dot(x1, x2)
        elif self.kernel == 'poly':
            return (np.dot(x1, x2) + 1) ** self.degree
        else:
            raise ValueError("Unsupported kernel type")

    def examine(self, i2):
        y2 = self.y[i2]
        alpha2 = self.alpha[i2]
        x2 = self.X[i2]
        E2 = self.decision_function(x2) - y2

       
        if (y2 * E2 < -self.tol and alpha2 < self.C) or (y2 * E2 > self.tol and alpha2 > 0):
            # Select i1 based on maximum error
            if E2 > 0:
                i1 = np.argmin(self.alpha)
            else:
                i1 = np.argmax(self.alpha)

            if self.smo_step(i1, i2):
                return 1

        return 0

    def smo_step(self, i1, i2):
        if i1 == i2:
            return 0

        alpha1 = self.alpha[i1]
        alpha2 = self.alpha[i2]
        y1 = self.y[i1]
        y2 = self.y[i2]
        x1 = self.X[i1]
        x2 = self.X[i2]

        E1 = self.decision_function(x1) - y1
        E2 = self.decision_function(x2) - y2

        # Compute L and H, the bounds on alpha2
        if y1 != y2:
            L = max(0, alpha2 - alpha1)
            H = min(self.C, self.C + alpha2 - alpha1)
        else:
            L = max(0, alpha1 + alpha2 - self.C)
            H = min(self.C, alpha1 + alpha2)

        if L == H:
            return 0

        k11 = self._kernel_function(x1, x1)
        k22 = self._kernel_function(x2, x2)
        k12 = self._kernel_function(x1, x2)

        # Compute eta
        eta = 2 * k12 - k11 - k22

        if eta < 0:
   
            alpha2_new = alpha2 - y2 * (E1 - E2) / eta
            alpha2_new = max(L, min(H, alpha2_new))

            if abs(alpha2_new - alpha2) < 1e-5:
                return 0

  
            alpha1_new = alpha1 + y1 * y2 * (alpha2 - alpha2_new)

        
            b1 = self.b - E1 - y1 * (alpha1_new - alpha1) * k11 - y2 * (alpha2_new - alpha2) * k12
            b2 = self.b - E2 - y1 * (alpha1_new - alpha1) * k12 - y2 * (alpha2_new - alpha2) * k22

            if 0 < alpha1_new < self.C:
                self.b = b1
            elif 0 < alpha2_new < self.C:
                self.b = b2
            else:
                self.b = (b1 + b2) / 2

           
            self.alpha[i1] = alpha1_new
            self.alpha[i2] = alpha2_new

            return 1

        return 0

    def decision_function(self, X):
        if len(X.shape) == 1:
            return np.sum(self.alpha * self.y * np.array([self._kernel_function(X, xi) for xi in self.X])) - self.b
        else:
            return np.sum(np.array([self.alpha * self.y * np.array([self._kernel_function(xi, xj) for xi in self.X]) for xj in X]) - self.b, axis=1)


    def _compute_bias(self):

        support_vector_indices = np.where((self.alpha > 0) & (self.alpha < self.C))[0]
        
     
        if len(support_vector_indices) > 0:
            i = support_vector_indices[0]
            return self.y[i] - np.sum(self.alpha * self.y * np.array([self._kernel_function(self.X[i], xi) for xi in self.X])) + self.b
        else:
            return 0

    def predict(self, X):
        if isinstance(X, np.ndarray):
            return np.sign(self.decision_function(X))
        else:
            return np.sign(self.decision_function(np.array(X)))

class MultiSVM():
    def __init__(self, kernel='linear', degree=3, C=1.0, tol=1e-3, max_iter=1000):
        self.kernel = kernel
        self.degree = degree
        self.C = C
        self.tol = tol
        self.max_iter = max_iter
        self.binary_classifiers = []

    def fit(self, X, y):
        self.unique_classes = np.unique(y)
        num_classes = len(self.unique_classes)

      
        for i in range(num_classes):
            for j in range(i + 1, num_classes):
              
                class_mask = (y == self.unique_classes[i]) | (y == self.unique_classes[j])
                X_binary = X[class_mask]
                y_binary = y[class_mask]

                binary_svm = SVM(kernel=self.kernel, degree=self.degree, C=self.C, tol=self.tol, max_iter=self.max_iter)
                binary_svm.fit(X_binary, y_binary)

                self.binary_classifiers.append(((self.unique_classes[i], self.unique_classes[j]), binary_svm))

    def predict(self, X):
        num_samples = X.shape[0]
        num_classes = len(self.unique_classes)
        votes = np.zeros((num_samples, num_classes), dtype=int)

        for binary_classes, binary_svm in self.binary_classifiers:
            i, j = binary_classes
            binary_predictions = binary_svm.predict(X)

            for k in range(num_samples):
                if binary_predictions[k] == 1:
                    votes[k, i] += 1
                else:
                    votes[k, j] += 1

   
        multi_class_predictions = np.argmax(votes, axis=1)

        return [self.unique_classes[i] for i in multi_class_predictions]

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target


scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Custom SVM with linear kernel
custom_linear_svm = SVM(kernel='linear', C=1.0, tol=1e-3, max_iter=1000)
custom_linear_svm.fit(X_train, y_train)
y_pred_custom_linear = custom_linear_svm.predict(X_test)
accuracy_custom_linear = accuracy_score(y_test, y_pred_custom_linear)
print("Custom Linear SVM:")
print(f"Accuracy: {accuracy_custom_linear:.2f}")

# Custom SVM with polynomial kernel
custom_poly_svm = SVM(kernel='poly', degree=3, C=1.0, tol=1e-3, max_iter=1000)
custom_poly_svm.fit(X_train, y_train)
y_pred_custom_poly = custom_poly_svm.predict(X_test)
accuracy_custom_poly = accuracy_score(y_test, y_pred_custom_poly)
print("Custom Polynomial SVM:")
print(f"Accuracy: {accuracy_custom_poly:.2f}")

# scikit-learn SVC with linear kernel
sklearn_linear_svm = SVC(kernel='linear', C=1.0)
sklearn_linear_svm.fit(X_train, y_train)
y_pred_sklearn_linear = sklearn_linear_svm.predict(X_test)
accuracy_sklearn_linear = accuracy_score(y_test, y_pred_sklearn_linear)
print("sklearn Linear SVM:")
print(f"Accuracy: {accuracy_sklearn_linear:.2f}")

# scikit-learn SVC with polynomial kernel
sklearn_poly_svm = SVC(kernel='poly', degree=3, C=1.0)
sklearn_poly_svm.fit(X_train, y_train)
y_pred_sklearn_poly = sklearn_poly_svm.predict(X_test)
accuracy_sklearn_poly = accuracy_score(y_test, y_pred_sklearn_poly)
print("sklearn Polynomial SVM:")
print(f"Accuracy: {accuracy_sklearn_poly:.2f}")










Custom Linear SVM:
Accuracy: 0.33
Custom Polynomial SVM:
Accuracy: 0.13
sklearn Linear SVM:
Accuracy: 0.97
sklearn Polynomial SVM:
Accuracy: 0.97
